In [91]:
!conda install -c conda-forge lat_lon_parser --y

Solving environment: ...working... done

# All requested packages already installed.

Retrieving notices: ...working... done




==> WARNING: A newer version of conda exists. <==
  current version: 4.14.0
  latest version: 23.7.4

Please update conda by running

    $ conda update -n base -c conda-forge conda




In [92]:
!pip install exif

In [93]:
!pip install pillow-heif

In [94]:
import os
import glob
import sys
from lat_lon_parser import parse
import geopandas as gpd
import pandas as pd
import exif
import pillow_heif
import numpy as np

from PIL import Image

In [95]:
# define fucntion to get file exif information
def get_exif(filename):
    image = Image.open(filename)
    image.verify()
    return image.getexif().get_ifd(0x8825)

# define fucntion to get location data from exif
def get_geotagging(exif):
    geo_tagging_info = {}
    # If no file image info return location data as (N:0, W:0)
    if not exif:
        return {'GPSLatitudeRef': 'N',
                'GPSLatitude': '(0, 0, 0)',
                'GPSLongitudeRef': 'W',
                'GPSLongitude': '(0, 0, 0)'}
    else:
        gps_keys = ['GPSVersionID', 'GPSLatitudeRef', 'GPSLatitude', 'GPSLongitudeRef', 'GPSLongitude',
                    'GPSAltitudeRef', 'GPSAltitude', 'GPSTimeStamp', 'GPSSatellites', 'GPSStatus', 'GPSMeasureMode',
                    'GPSDOP', 'GPSSpeedRef', 'GPSSpeed', 'GPSTrackRef', 'GPSTrack', 'GPSImgDirectionRef',
                    'GPSImgDirection', 'GPSMapDatum', 'GPSDestLatitudeRef', 'GPSDestLatitude', 'GPSDestLongitudeRef',
                    'GPSDestLongitude', 'GPSDestBearingRef', 'GPSDestBearing', 'GPSDestDistanceRef', 'GPSDestDistance',
                    'GPSProcessingMethod', 'GPSAreaInformation', 'GPSDateStamp', 'GPSDifferential']

        for k, v in exif.items():
            try:
                geo_tagging_info[gps_keys[k]] = str(v)
            except IndexError:
                pass
        return geo_tagging_info


In [96]:
# set file path to image directory (downlaoded from one drive shared file)
path = "C:/Users/mediwils/Downloads/ImagesFromLeedsLSOAs/"

# Problem with backslash: "\\"
# cerate empty list of dfs
dfs= []
master_df =pd.DataFrame()
paths =os.listdir(path)

# For sub-directory in the full directory
for p in paths:
    for root, dirs, files in os.walk(path+p):
        # Get the name of all files in directory
        for name in files:
            # get jpg files
            if name.endswith(".JPG"):
                file_path = os.path.join(root, name)
                image_info = get_exif(file_path)
                results = get_geotagging(image_info)
                df=pd.DataFrame(columns =['IMGid','Latitude','Longitude'])
                df.loc[0]= [name.rstrip(".JPG"),parse(results['GPSLatitude']),(parse(results['GPSLongitude'])*-1)]
                df["file_path"] = str(file_path)
                master_df =pd.concat([master_df,df])
            # if not jpg get jpeg files
            elif name.endswith(".jpeg"):
                file_path = os.path.join(root, name)
                image_info = get_exif(file_path)
                results = get_geotagging(image_info)
                df=pd.DataFrame(columns =['IMGid','Latitude','Longitude'])
                df.loc[0]= [name.rstrip(".jpeg"),parse(results['GPSLatitude']),(parse(results['GPSLongitude'])*-1)]
                df["file_path"] = str(file_path)
                master_df =pd.concat([master_df,df])  
            elif name.endswith(".jpg"):
                file_path = os.path.join(root, name)
                image_info = get_exif(file_path)
                results = get_geotagging(image_info)
                df=pd.DataFrame(columns =['IMGid','Latitude','Longitude'])
                df.loc[0]= [name.rstrip(".jpg"),parse(results['GPSLatitude']),(parse(results['GPSLongitude'])*-1)]
                df["file_path"] = str(file_path)
                master_df =pd.concat([master_df,df])
            #    df[["Latitude","Longitude"]] = df[["Latitude","Longitude"]].fillna(0)
master_df =pd.concat([master_df,df])  

In [97]:
name

'20230523_115524.jpg'

In [98]:
master_df.head()

,IMGid,Latitude,Longitude,file_path
0,IMG_5123,53.805711,-1.628422,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...
0,IMG_5124,53.805664,-1.628519,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...
0,IMG_5125,53.805928,-1.632267,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...
0,IMG_5126,53.805928,-1.632200,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...
0,IMG_5127,53.805908,-1.632492,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...


In [99]:
# Get LSOA quantile and other info from file path
master_df['LSOA_naming'] =master_df['file_path'].str.split('/', expand=True)[5]
master_df['LSOA_naming'] =master_df['LSOA_naming'].str.split("\\", expand=True)[0]
master_df[['lsoa_code', 'IMD_Q','LSOA_name']]= master_df['LSOA_naming'].str.split('_', expand=True)
# master_df['lsoa_code']= master_df['LSOA_naming'].str.split('_', expand=True)[0]
# master_df['IMD_Q']= master_df['LSOA_naming'].str.split('_', expand=True)[1]
# master_df['LSOA_name']= master_df['LSOA_naming'].str.split('_', expand=True)[2]
master_df['LSOA_name'] = master_df['LSOA_name'].str.replace( r"([A-Z])", r" \1").str.strip()

                                                            # If no location info reaplce with Nan
master_df.loc[(master_df['Latitude']==0),'Longitude']=np.NaN
master_df.loc[(master_df['Latitude']==0),'Latitude']=np.NaN

# Convert to geodataframe 
#master_gdf =gpd.GeoDataFrame(master_df,geometry=gpd.points_from_xy(master_df.Longitude, master_df.Latitude))
# Set CRS
#master_gdf =master_gdf.set_crs("EPSG:4326")

In [100]:
master_df.tail()

,IMGid,Latitude,Longitude,file_path,LSOA_naming,lsoa_code,IMD_Q,LSOA_name
0,20230523_113538,53.797741,-1.522829,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01033021_Q2_UpperAccommodationRd,E01033021,Q2,UpperAccommodationRd
0,20230523_113831,53.797844,-1.520840,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01033021_Q2_UpperAccommodationRd,E01033021,Q2,UpperAccommodationRd
0,20230523_115503,53.795664,-1.516769,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01033021_Q2_UpperAccommodationRd,E01033021,Q2,UpperAccommodationRd
0,20230523_115524,53.795188,-1.517370,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01033021_Q2_UpperAccommodationRd,E01033021,Q2,UpperAccommodationRd
0,20230523_115524,53.795188,-1.517370,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01033021_Q2_UpperAccommodationRd,E01033021,Q2,UpperAccommodationRd


In [101]:
master_df.to_csv('master_df.csv')

In [102]:
#look at geo-data frame
master_gdf

NameError: name 'master_gdf' is not defined

In [103]:
AdvertData = pd.read_excel(r"C:\Users\mediwils\Downloads\AdvertData.xlsx")

In [104]:
AdvertData_df= AdvertData[["LSOA", "IMGid"]]

In [105]:
#display(AdvertData_df.to_string())

In [106]:
AdvertData_df

,LSOA,IMGid
0,E01011597,IMG_4652.JPG
1,E01011597,IMG_4653.JPG
2,E01011597,IMG_4657.JPG
3,E01011597,IMG_4658.JPG
4,E01011597,IMG_4659.JPG
...,...,...
921,E01011271,20230516_162732.jpg
922,E01011271,20230516_162732.jpg
923,E01011271,20230516_165621.jpg
924,E01011271,20230516_165621.jpg


In [107]:
merged_df = pd.read_csv("adData_master_df_joined.csv")

In [108]:
merged_df

,Unnamed: 0,Rinitials,Cinitials,Codedate,LSOA,IMGid,IMGdate,ADid,ASSETid,Long,...,NPMstatus,ADcompliance,...1,Latitude,Longitude,file_path,LSOA_naming,lsoa_code,IMD_Q,LSOA_name
0,898,MM,IW,20/06/2023,E01011267,20230518_075121,18/05/2023,GR_WQGB_03_01,GR_WQGB_03,NaN,...,NaN,NaN,0.0,53.870589,-1.705423,C:\Users\mediwils\OneDrive - University of Lee...,E01011267_Q3_WestQueenswayGreenbottom,E01011267,Q3,WestQueenswayGreenbottom
1,899,MM,IW,20/06/2023,E01011267,20230518_075130,18/05/2023,GR_WQGB_04_01,GR_WQGB_04,NaN,...,NaN,NaN,0.0,53.870589,-1.705423,C:\Users\mediwils\OneDrive - University of Lee...,E01011267_Q3_WestQueenswayGreenbottom,E01011267,Q3,WestQueenswayGreenbottom
2,248,VJ,VJ,23/05/2023,E01011397,20230523_093239,23/05/2023,GS_MRFF_22_03,GS_MRFF_22,NaN,...,NaN,NaN,0.0,53.791983,-1.368975,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01011397_Q5_MeadowRoadFirthfieldsEastGarforth,E01011397,Q5,MeadowRoadFirthfieldsEastGarforth
3,246,VJ,VJ,23/05/2023,E01011397,20230523_093248,23/05/2023,GS_MRFF_22_01,GS_MRFF_22,NaN,...,NaN,NaN,0.0,53.792006,-1.368991,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01011397_Q5_MeadowRoadFirthfieldsEastGarforth,E01011397,Q5,MeadowRoadFirthfieldsEastGarforth
4,247,VJ,VJ,23/05/2023,E01011397,20230523_093242,23/05/2023,GS_MRFF_22_02,GS_MRFF_22,NaN,...,NaN,NaN,0.0,53.791954,-1.369008,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01011397_Q5_MeadowRoadFirthfieldsEastGarforth,E01011397,Q5,MeadowRoadFirthfieldsEastGarforth
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
922,924,MM,IW,20/06/2023,E01011271,20230516_162732,16/05/2023,GR_FHHS_05_03,GR_FHHS_05,NaN,...,NaN,NaN,0.0,53.875301,-1.717322,C:\Users\mediwils\OneDrive - University of Lee...,E01011271_Q5_FieldheadHawkstones,E01011271,Q5,FieldheadHawkstones
923,927,MM,IW,20/06/2023,E01011271,20230516_171251,16/05/2023,GR_FHHS_07_01,GR_FHHS_07,NaN,...,NaN,NaN,0.0,53.873121,-1.718240,C:\Users\mediwils\OneDrive - University of Lee...,E01011271_Q5_FieldheadHawkstones,E01011271,Q5,FieldheadHawkstones
924,925,MM,IW,20/06/2023,E01011271,20230516_165621,16/05/2023,GR_FHHS_06_01,GR_FHHS_06,NaN,...,NaN,NaN,0.0,53.868420,-1.722792,C:\Users\mediwils\OneDrive - University of Lee...,E01011271_Q5_FieldheadHawkstones,E01011271,Q5,FieldheadHawkstones
925,926,MM,IW,20/06/2023,E01011271,20230516_165621,16/05/2023,GR_FHHS_06_02,GR_FHHS_06,NaN,...,NaN,NaN,0.0,53.868420,-1.722792,C:\Users\mediwils\OneDrive - University of Lee...,E01011271_Q5_FieldheadHawkstones,E01011271,Q5,FieldheadHawkstones


In [109]:
count_nan = merged_df['Latitude'].isnull().sum()
count_nan

0

In [110]:
# Convert to geodataframe 
merged_gdf =gpd.GeoDataFrame(merged_df,geometry=gpd.points_from_xy(merged_df.Longitude, merged_df.Latitude))
# Set CRS
merged_gdf =merged_gdf.set_crs("EPSG:4326")

In [111]:
merged_gdf.head()

,Unnamed: 0,Rinitials,Cinitials,Codedate,LSOA,IMGid,IMGdate,ADid,ASSETid,Long,...,ADcompliance,...1,Latitude,Longitude,file_path,LSOA_naming,lsoa_code,IMD_Q,LSOA_name,geometry
0,898,MM,IW,20/06/2023,E01011267,20230518_075121,18/05/2023,GR_WQGB_03_01,GR_WQGB_03,NaN,...,NaN,0.0,53.870589,-1.705423,C:\Users\mediwils\OneDrive - University of Lee...,E01011267_Q3_WestQueenswayGreenbottom,E01011267,Q3,WestQueenswayGreenbottom,POINT (-1.70542 53.87059)
1,899,MM,IW,20/06/2023,E01011267,20230518_075130,18/05/2023,GR_WQGB_04_01,GR_WQGB_04,NaN,...,NaN,0.0,53.870589,-1.705423,C:\Users\mediwils\OneDrive - University of Lee...,E01011267_Q3_WestQueenswayGreenbottom,E01011267,Q3,WestQueenswayGreenbottom,POINT (-1.70542 53.87059)
2,248,VJ,VJ,23/05/2023,E01011397,20230523_093239,23/05/2023,GS_MRFF_22_03,GS_MRFF_22,NaN,...,NaN,0.0,53.791983,-1.368975,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01011397_Q5_MeadowRoadFirthfieldsEastGarforth,E01011397,Q5,MeadowRoadFirthfieldsEastGarforth,POINT (-1.36897 53.79198)
3,246,VJ,VJ,23/05/2023,E01011397,20230523_093248,23/05/2023,GS_MRFF_22_01,GS_MRFF_22,NaN,...,NaN,0.0,53.792006,-1.368991,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01011397_Q5_MeadowRoadFirthfieldsEastGarforth,E01011397,Q5,MeadowRoadFirthfieldsEastGarforth,POINT (-1.36899 53.79201)
4,247,VJ,VJ,23/05/2023,E01011397,20230523_093242,23/05/2023,GS_MRFF_22_02,GS_MRFF_22,NaN,...,NaN,0.0,53.791954,-1.369008,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01011397_Q5_MeadowRoadFirthfieldsEastGarforth,E01011397,Q5,MeadowRoadFirthfieldsEastGarforth,POINT (-1.36901 53.79195)


In [112]:
#unknown_coord = merged_df.loc[merged_df['IMGid']=='20230522_103337', ['Latitude','Longitude']]
#master_df.loc[merged_df['IMGid']=='20230522_103324', ['Latitude','Longitude']] = unknown_coord

In [113]:
df= AreaAttributes = pd.read_excel(r"C:\Users\mediwils\Downloads\AreaAttributes.xlsx")

In [114]:
# Look at geo-data frame
#master_gdf

In [115]:
#display(master_df.to_string())

In [116]:
# # svae file info to excel doc
merged_gdf.to_excel("Image_location_info.xlsx",sheet_name='Image_locations')  

# AdvertData =pd.read_excel('/Users/francescapontin/Downloads/AdvertData.xlsx')

In [117]:
merged_gdf.head()

,Unnamed: 0,Rinitials,Cinitials,Codedate,LSOA,IMGid,IMGdate,ADid,ASSETid,Long,...,ADcompliance,...1,Latitude,Longitude,file_path,LSOA_naming,lsoa_code,IMD_Q,LSOA_name,geometry
0,898,MM,IW,20/06/2023,E01011267,20230518_075121,18/05/2023,GR_WQGB_03_01,GR_WQGB_03,NaN,...,NaN,0.0,53.870589,-1.705423,C:\Users\mediwils\OneDrive - University of Lee...,E01011267_Q3_WestQueenswayGreenbottom,E01011267,Q3,WestQueenswayGreenbottom,POINT (-1.70542 53.87059)
1,899,MM,IW,20/06/2023,E01011267,20230518_075130,18/05/2023,GR_WQGB_04_01,GR_WQGB_04,NaN,...,NaN,0.0,53.870589,-1.705423,C:\Users\mediwils\OneDrive - University of Lee...,E01011267_Q3_WestQueenswayGreenbottom,E01011267,Q3,WestQueenswayGreenbottom,POINT (-1.70542 53.87059)
2,248,VJ,VJ,23/05/2023,E01011397,20230523_093239,23/05/2023,GS_MRFF_22_03,GS_MRFF_22,NaN,...,NaN,0.0,53.791983,-1.368975,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01011397_Q5_MeadowRoadFirthfieldsEastGarforth,E01011397,Q5,MeadowRoadFirthfieldsEastGarforth,POINT (-1.36897 53.79198)
3,246,VJ,VJ,23/05/2023,E01011397,20230523_093248,23/05/2023,GS_MRFF_22_01,GS_MRFF_22,NaN,...,NaN,0.0,53.792006,-1.368991,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01011397_Q5_MeadowRoadFirthfieldsEastGarforth,E01011397,Q5,MeadowRoadFirthfieldsEastGarforth,POINT (-1.36899 53.79201)
4,247,VJ,VJ,23/05/2023,E01011397,20230523_093242,23/05/2023,GS_MRFF_22_02,GS_MRFF_22,NaN,...,NaN,0.0,53.791954,-1.369008,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01011397_Q5_MeadowRoadFirthfieldsEastGarforth,E01011397,Q5,MeadowRoadFirthfieldsEastGarforth,POINT (-1.36901 53.79195)


In [118]:
# list columns in advert data 
AdvertData.columns

Index(['Rinitials', 'Cinitials', 'Codedate', 'LSOA', 'IMGid', 'IMGdate',
       'ADid', 'ASSETid', 'Long', 'Lat', 'ADtype', 'ADsize', 'ADprodtype',
       'ADprod', 'ADbrand', 'Brandad', 'Admanagement', 'Price£',
       'Pricesourcelink', 'Unit', 'Portion', 'Totalwt', 'Wtsource',
       'Nutritionsource', 'Nutsourcelink', 'Fatdensity', 'Satfatdensity',
       'Sugardensity', 'Sodiumdensity', 'Ekcaldensity', 'Fibredensity',
       'Proteindensity', 'FVN', 'NPMscore', 'NPMstatus', 'ADcompliance'],
      dtype='object')

In [119]:
# remove file extension to get an image name
AdvertData['IMGid'] =AdvertData['IMGid'].str.strip('.JPG').str.strip('.jpg')

# Get a count of Image locaiton data from files that are in the master spreadsheet
merged_gdf['IMGid'].isin(AdvertData['IMGid'].str.strip('.JPG').str.strip('.jpg')).value_counts()

IMGid
True     925
False      2
Name: count, dtype: int64

In [120]:
# remove duplicates
AdvertData['duplicate_image_lsoa'] =AdvertData.duplicated(subset=['IMGid','LSOA'], keep=False)

In [121]:
AdvertData.loc[AdvertData['duplicate_image_lsoa']==True,:]

,Rinitials,Cinitials,Codedate,LSOA,IMGid,IMGdate,ADid,ASSETid,Long,Lat,...,Sugardensity,Sodiumdensity,Ekcaldensity,Fibredensity,Proteindensity,FVN,NPMscore,NPMstatus,ADcompliance,duplicate_image_lsoa
5,IW,IW,2023-05-17,E01011597,IMG_4660,2023-05-11,CF_GLMR_06_01,CF_GLMR_06,NaN,NaN,...,2.61,1033.00,312.70,1.3,17.51,0.0,20.0,1.0,1.0,True
6,IW,IW,2023-05-17,E01011597,IMG_4660,2023-05-11,CF_GLMR_06_01,CF_GLMR_06,NaN,NaN,...,4.70,44.00,54.00,0.0,4.10,0.0,0.0,0.0,0.0,True
19,IW,IW,2023-05-17,E01011597,IMG_4651,2023-05-11,CF_GLMR_14_01,CF_GLMR_14,NaN,NaN,...,0.00,4.00,117.00,0.6,2.80,0.0,0.0,0.0,0.0,True
20,IW,IW,2023-05-17,E01011597,IMG_4651,2023-05-11,CF_GLMR_14_01,CF_GLMR_14,NaN,NaN,...,2.50,320.00,72.00,1.5,10.70,0.0,-3.0,0.0,0.0,True
21,IW,IW,2023-05-17,E01011597,IMG_4677,2023-05-11,CF_GLMR_15_01,CF_GLMR_15,NaN,NaN,...,0.15,153.85,192.31,0.0,2.69,0.0,2.0,0.0,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
920,MM,IW,2023-06-20,E01011271,20230516_162732,2023-05-16,GR_FHHS_05_01,GR_FHHS_05,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
921,MM,IW,2023-06-20,E01011271,20230516_162732,2023-05-16,GR_FHHS_05_02,GR_FHHS_05,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
922,MM,IW,2023-06-20,E01011271,20230516_162732,2023-05-16,GR_FHHS_05_03,GR_FHHS_05,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
923,MM,IW,2023-06-20,E01011271,20230516_165621,2023-05-16,GR_FHHS_06_01,GR_FHHS_06,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [122]:
# Add in validation step once data pre-processing complete
#a.merge(master_gdf, left_on=['IMGname','LSOA'], right_on=['IMGname','lsoa_code'], how='left',validate="1:1")

In [123]:
Add_gdf =gpd.GeoDataFrame(merged_gdf)

In [124]:
#Add_gdf =gpd.GeoDataFrame(AdvertData.merge(merged_gdf, left_on=['IMGid','LSOA'], right_on=['IMGid','lsoa_code'], how='left', indicator=True))

In [125]:
Add_gdf

,Unnamed: 0,Rinitials,Cinitials,Codedate,LSOA,IMGid,IMGdate,ADid,ASSETid,Long,...,ADcompliance,...1,Latitude,Longitude,file_path,LSOA_naming,lsoa_code,IMD_Q,LSOA_name,geometry
0,898,MM,IW,20/06/2023,E01011267,20230518_075121,18/05/2023,GR_WQGB_03_01,GR_WQGB_03,NaN,...,NaN,0.0,53.870589,-1.705423,C:\Users\mediwils\OneDrive - University of Lee...,E01011267_Q3_WestQueenswayGreenbottom,E01011267,Q3,WestQueenswayGreenbottom,POINT (-1.70542 53.87059)
1,899,MM,IW,20/06/2023,E01011267,20230518_075130,18/05/2023,GR_WQGB_04_01,GR_WQGB_04,NaN,...,NaN,0.0,53.870589,-1.705423,C:\Users\mediwils\OneDrive - University of Lee...,E01011267_Q3_WestQueenswayGreenbottom,E01011267,Q3,WestQueenswayGreenbottom,POINT (-1.70542 53.87059)
2,248,VJ,VJ,23/05/2023,E01011397,20230523_093239,23/05/2023,GS_MRFF_22_03,GS_MRFF_22,NaN,...,NaN,0.0,53.791983,-1.368975,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01011397_Q5_MeadowRoadFirthfieldsEastGarforth,E01011397,Q5,MeadowRoadFirthfieldsEastGarforth,POINT (-1.36897 53.79198)
3,246,VJ,VJ,23/05/2023,E01011397,20230523_093248,23/05/2023,GS_MRFF_22_01,GS_MRFF_22,NaN,...,NaN,0.0,53.792006,-1.368991,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01011397_Q5_MeadowRoadFirthfieldsEastGarforth,E01011397,Q5,MeadowRoadFirthfieldsEastGarforth,POINT (-1.36899 53.79201)
4,247,VJ,VJ,23/05/2023,E01011397,20230523_093242,23/05/2023,GS_MRFF_22_02,GS_MRFF_22,NaN,...,NaN,0.0,53.791954,-1.369008,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01011397_Q5_MeadowRoadFirthfieldsEastGarforth,E01011397,Q5,MeadowRoadFirthfieldsEastGarforth,POINT (-1.36901 53.79195)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
922,924,MM,IW,20/06/2023,E01011271,20230516_162732,16/05/2023,GR_FHHS_05_03,GR_FHHS_05,NaN,...,NaN,0.0,53.875301,-1.717322,C:\Users\mediwils\OneDrive - University of Lee...,E01011271_Q5_FieldheadHawkstones,E01011271,Q5,FieldheadHawkstones,POINT (-1.71732 53.87530)
923,927,MM,IW,20/06/2023,E01011271,20230516_171251,16/05/2023,GR_FHHS_07_01,GR_FHHS_07,NaN,...,NaN,0.0,53.873121,-1.718240,C:\Users\mediwils\OneDrive - University of Lee...,E01011271_Q5_FieldheadHawkstones,E01011271,Q5,FieldheadHawkstones,POINT (-1.71824 53.87312)
924,925,MM,IW,20/06/2023,E01011271,20230516_165621,16/05/2023,GR_FHHS_06_01,GR_FHHS_06,NaN,...,NaN,0.0,53.868420,-1.722792,C:\Users\mediwils\OneDrive - University of Lee...,E01011271_Q5_FieldheadHawkstones,E01011271,Q5,FieldheadHawkstones,POINT (-1.72279 53.86842)
925,926,MM,IW,20/06/2023,E01011271,20230516_165621,16/05/2023,GR_FHHS_06_02,GR_FHHS_06,NaN,...,NaN,0.0,53.868420,-1.722792,C:\Users\mediwils\OneDrive - University of Lee...,E01011271_Q5_FieldheadHawkstones,E01011271,Q5,FieldheadHawkstones,POINT (-1.72279 53.86842)


In [126]:
# re-format datetime to avoid explore() potting issues
Add_gdf_plot =Add_gdf
Add_gdf_plot['Codedate'] =Add_gdf['Codedate'].astype(str)
Add_gdf_plot['IMGdate'] =Add_gdf['IMGdate'].astype(str)

In [127]:
# For now only plot images with locaiton info
Add_gdf_plot=Add_gdf_plot.dropna(subset=['geometry'])

In [128]:
Add_gdf_plot.to_excel("Advert_image_location_info.xlsx",sheet_name='Image_locations')  

In [129]:
Add_gdf_plot.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 927 entries, 0 to 926
Data columns (total 46 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   Unnamed: 0       927 non-null    int64   
 1   Rinitials        927 non-null    object  
 2   Cinitials        927 non-null    object  
 3   Codedate         927 non-null    object  
 4   LSOA             927 non-null    object  
 5   IMGid            927 non-null    object  
 6   IMGdate          927 non-null    object  
 7   ADid             927 non-null    object  
 8   ASSETid          927 non-null    object  
 9   Long             0 non-null      float64 
 10  Lat              0 non-null      float64 
 11  ADtype           926 non-null    float64 
 12  ADsize           926 non-null    float64 
 13  ADprodtype       920 non-null    float64 
 14  ADprod           920 non-null    object  
 15  ADbrand          903 non-null    object  
 16  Brandad          917 non-null    flo

In [130]:
# Also save as geojson
Add_gdf_plot.to_file("Advert_image_location_info.geojson", driver='GeoJSON')

In [131]:
# Also save as geojson
#Add_gdf_plot.drop(columns='_merge').to_file("Advert_image_location_info.geojson", driver='GeoJSON')